# Web scrapping TripAdvisor-Mexico

Importing necessary libraries:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#A list to store the scraped data
cities_list = []

Functions to scrapp the cities:

In [3]:
def formula(num:int) -> int:

    return int(num-1)*20

def extract(page) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'}
    
    url = f'https://www.tripadvisor.com.mx/Restaurants-g150768-oa{page}-Mexico.html#LOCATION_LIST'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup


def transform(soup) -> list:
    
    divs = soup.find_all('div', class_='geo_wrap')

    for item in divs:
        city =  item.find('a').text
        href =  item.find('a')['href']

        city ={'city': city, 'href': href}
        cities_list.append(city)

    return  cities_list

def transform2(soup) -> list:
    
    for ultag in soup.find_all('ul', {'class': 'geoList'}):
        for litag in ultag.find_all('li'):
            city= litag.text
            href= litag.find('a')['href']
            city = {'city':city,'href':href}
            cities_list.append(city)



In [4]:
transform(extract(formula(1)))

[{'city': 'Restaurantes en Ciudad de México',
  'href': '/Restaurants-g150800-Mexico_City_Central_Mexico_and_Gulf_Coast.html'},
 {'city': 'Restaurantes en Guadalajara',
  'href': '/Restaurants-g150798-Guadalajara_Guadalajara_Metropolitan_Area.html'},
 {'city': 'Restaurantes en Cancún',
  'href': '/Restaurants-g150807-Cancun_Yucatan_Peninsula.html'},
 {'city': 'Restaurantes en Playa del Carmen',
  'href': '/Restaurants-g150812-Playa_del_Carmen_Yucatan_Peninsula.html'},
 {'city': 'Restaurantes en Monterrey',
  'href': '/Restaurants-g150782-Monterrey_Northern_Mexico.html'},
 {'city': 'Restaurantes en Puerto Vallarta',
  'href': '/Restaurants-g150793-Puerto_Vallarta.html'},
 {'city': 'Restaurantes en Santiago de Querétaro',
  'href': '/Restaurants-g479232-Queretaro_City_Central_Mexico_and_Gulf_Coast.html'},
 {'city': 'Restaurantes en Mérida',
  'href': '/Restaurants-g150811-Merida_Yucatan_Peninsula.html'},
 {'city': 'Restaurantes en Puebla',
  'href': '/Restaurants-g152773-Puebla_Central_M

In [5]:
# city_list = []
for i in range(2,3):
    transform2(extract(formula(i)))

In [6]:
df_cities = pd.DataFrame(cities_list)

In [7]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   city    40 non-null     object
 1   href    40 non-null     object
dtypes: object(2)
memory usage: 768.0+ bytes


In [8]:
df_cities['city'] = df_cities['city'].str.lstrip('Restaurantes en ')

In [9]:
df_cities['state'] =''
for i in range(len(df_cities)):
    if '\n-' in df_cities['city'][i]:
        df_cities['state'][i] = df_cities['city'][i].split('\n-')[1]
        df_cities['city'][i] = df_cities['city'][i].split('\n-')[0]

In [10]:
df_cities.head(20)

,city,href,state
0,Ciudad de México,/Restaurants-g150800-Mexico_City_Central_Mexic...,
1,Guadalajara,/Restaurants-g150798-Guadalajara_Guadalajara_M...,
2,Cancún,/Restaurants-g150807-Cancun_Yucatan_Peninsula....,
3,Playa del Carmen,/Restaurants-g150812-Playa_del_Carmen_Yucatan_...,
4,Monterrey,/Restaurants-g150782-Monterrey_Northern_Mexico...,
5,Puerto Vallarta,/Restaurants-g150793-Puerto_Vallarta.html,
6,Santiago de Querétaro,/Restaurants-g479232-Queretaro_City_Central_Me...,
7,Mérida,/Restaurants-g150811-Merida_Yucatan_Peninsula....,
8,Puebla,/Restaurants-g152773-Puebla_Central_Mexico_and...,
9,Zapopan,/Restaurants-g1006488-Zapopan_Guadalajara_Metr...,


In [11]:
#df_cities.to_csv('city_list.csv', index=False)

Functions to scrap the restaurants name and webpage:

In [12]:
def extractR(page) -> BeautifulSoup:
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'}
    
    url = f'https://www.tripadvisor.com.mx{page}'
    r = requests.get(url, headers=headers)

    soup = BeautifulSoup(r.content, 'html.parser')

    return soup,page

restaurants_list = []

def transformR(soup,page) -> list:
    
    divs = soup.find_all('div', class_='zdCeB Vt o')

    for item in divs:
        if '.' in item.find('a', class_='Lwqic Cj b').text:

            restaurant =  item.find('a', class_='Lwqic Cj b').text.split('.')[1]
            href = item.find('a', class_='Lwqic Cj b')['href']
            

            restaurants ={'id':page,
                          'restaurant': restaurant, 
                          'href':href}
                        
            restaurants_list.append(restaurants)


In [13]:
for href in df_cities['href']:
    soup,page = extractR(href)
    transformR(soup,page)

In [14]:
df_restaurants = pd.DataFrame(restaurants_list )

In [15]:
df_restaurants

,id,restaurant,href
0,/Restaurants-g150800-Mexico_City_Central_Mexic...,Balta,/Restaurant_Review-g150800-d19338104-Reviews-B...
1,/Restaurants-g150800-Mexico_City_Central_Mexic...,La Mansion Marriott Reforma,/Restaurant_Review-g150800-d2394477-Reviews-La...
2,/Restaurants-g150800-Mexico_City_Central_Mexic...,Condimento Restaurant,/Restaurant_Review-g150800-d1926008-Reviews-Co...
3,/Restaurants-g150800-Mexico_City_Central_Mexic...,La Distral,/Restaurant_Review-g150800-d12104177-Reviews-L...
4,/Restaurants-g150800-Mexico_City_Central_Mexic...,Restaurante Condimento Emporio Reforma,/Restaurant_Review-g150800-d22879733-Reviews-R...
...,...,...,...
1196,/Restaurants-g153373-Puerto_Escondido_Southern...,Dan's Cafe Deluxe,/Restaurant_Review-g153373-d2418201-Reviews-Da...
1197,/Restaurants-g153373-Puerto_Escondido_Southern...,Benditos Ristorante Pizzeria Italiana,/Restaurant_Review-g153373-d7322957-Reviews-Be...
1198,/Restaurants-g153373-Puerto_Escondido_Southern...,Kuhl Frozen Yogurt,/Restaurant_Review-g153373-d3710549-Reviews-Ku...
1199,/Restaurants-g153373-Puerto_Escondido_Southern...,Sativa Puerto,/Restaurant_Review-g153373-d1180559-Reviews-Sa...


In [16]:
df_restaurants = df_restaurants.merge(df_cities, left_on='id', right_on='href')
df_restaurants.drop(columns=['id','href_y'], inplace=True)
df_restaurants.rename(columns={'href_x':'href'}, inplace=True)

In [17]:
df_restaurants

,restaurant,href,city,state
0,Balta,/Restaurant_Review-g150800-d19338104-Reviews-B...,Ciudad de México,
1,La Mansion Marriott Reforma,/Restaurant_Review-g150800-d2394477-Reviews-La...,Ciudad de México,
2,Condimento Restaurant,/Restaurant_Review-g150800-d1926008-Reviews-Co...,Ciudad de México,
3,La Distral,/Restaurant_Review-g150800-d12104177-Reviews-L...,Ciudad de México,
4,Restaurante Condimento Emporio Reforma,/Restaurant_Review-g150800-d22879733-Reviews-R...,Ciudad de México,
...,...,...,...,...
1196,Dan's Cafe Deluxe,/Restaurant_Review-g153373-d2418201-Reviews-Da...,Puerto Escondido,Oaxaca
1197,Benditos Ristorante Pizzeria Italiana,/Restaurant_Review-g153373-d7322957-Reviews-Be...,Puerto Escondido,Oaxaca
1198,Kuhl Frozen Yogurt,/Restaurant_Review-g153373-d3710549-Reviews-Ku...,Puerto Escondido,Oaxaca
1199,Sativa Puerto,/Restaurant_Review-g153373-d1180559-Reviews-Sa...,Puerto Escondido,Oaxaca


Functions to scrap the restaurants info:

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [27]:
def extractRI(page) -> BeautifulSoup:

    CHROMEDRIVER_PATH = r"/Users/rosaarzabala/Documents/Projects/TripAdvisorScrapper/chromedriver"
    
    url = f'https://www.tripadvisor.com.mx{page}'
    useragent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0'

    options = Options()
    options.headless = True

    driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)
    driver.get(url)

    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')

    return soup

restaurants_info_list = []

In [23]:
def transformRI(soup,restaurant,city,state) -> list:
    
    calification = ''
    number_of_reviwes = ''
    ranking = ''
    address = ''
    web_site = ''
    latitude = ''
    longitude = ''

    div = soup.find('div', class_='YDAvY R2 F1 e k')
    for item in div:
        try:
            calification = item.find('span', class_='ZDEqb').text
            # print('calification:',calification)
        except:
            pass

        try:
            number_of_reviwes = item.find('a', class_='IcelI').text.rstrip(' opiniones')
            # print('number_of_reviwes:',number_of_reviwes)
        except:
            pass


    div = soup.find_all('div', class_='vQlTa H3')
    for item in div:
        try:
            ranking = item.find('span', class_='DsyBj cNFrA').text.split(' ')[0]
            # print('ranking:',ranking)
            break
        except:
            pass


    div = soup.find('div', class_='kDZhm IdiaP Me')
    for item in div:
        try:
            address = item.find('span', class_='yEWoV').text
            # print('address:',address)
        except:
            pass


    div = soup.find('div', class_='IdiaP Me sNsFa')
    for item in div:
        try:
            web_site = item.find('a', class_='YnKZo Ci Wc _S C FPPgD')['href']
            # print('web_site:',web_site)
        except:
            pass


    items = soup.find_all('div', class_='kDZhm IdiaP Me')
    for item in items:
        try:
            coordinates = item.find('a', class_='YnKZo Ci Wc _S C FPPgD')['href']
            coordinates = coordinates.split('@')[-1]
            latitude = coordinates.split(',')[0]
            longitude = coordinates.split(',')[1]
            # print('latitude:',latitude)
            # print('longite:',longitude)
        except:
            pass


    restaurants_info= {'city':city,
                       'state':state,
                       'restaurant':restaurant,
                       'calification': calification,
                       'number_of_reviwes': number_of_reviwes,
                       'ranking': ranking,
                       'address': address,
                       'web_site': web_site,
                       'latitude': latitude,
                       'longitude': longitude}

    restaurants_info_list.append(restaurants_info)


In [28]:
for i in range(2):
    href,restaurant,city,state = df_restaurants['href'][i],df_restaurants['restaurant'][i],df_restaurants['city'][i],df_restaurants['state'][i]
    transformRI(extractRI(href), restaurant, city, state)

/var/folders/5w/5qvg4ht11gqbxjjz9hc2zbs80000gn/T/ipykernel_18264/802171397.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)


In [29]:
df_restaurants_info = pd.DataFrame(restaurants_info_list)

In [30]:
df_restaurants_info

,city,state,restaurant,calification,number_of_reviwes,ranking,address,web_site,latitude,longitude
0,Ciudad de México,,Balta,5.0,479,#1,Avenida Paseo de La Reforma 297 Colonia Cuauht...,http://www.sofitel-mexico-city.com/restaurants...,19.428432,-99.16592
1,Ciudad de México,,La Mansion Marriott Reforma,5.0,992,#2,"Paseo de la Reforma 276 Col. Juarez, Mexico Ci...",http://www.facebook.com/mansionmarriottreforma,19.428345,-99.16426


In [31]:
df_cities['href'][:4]

0    /Restaurants-g150800-Mexico_City_Central_Mexic...
1    /Restaurants-g150798-Guadalajara_Guadalajara_M...
2    /Restaurants-g150807-Cancun_Yucatan_Peninsula....
3    /Restaurants-g150812-Playa_del_Carmen_Yucatan_...
Name: href, dtype: object